# Предсказание цены на автомобили

Датасет содержит 10 000 записей, созданных с целью прогнозирования цен на автомобили. Каждая строка представляет информацию об автомобиле и его цене.

**Описание столбцов:**

- *Brand*: марка автомобиля.

- *Model*: модель автомобиля.

- *Year*: год выпуска.

- *Engine_Size*: размер двигателя в литрах.

- *Fuel_Type*: тип топлива, используемого автомобилем:
    - *Petrol*: автомобили, работающие на бензине.
    - *Diesel*: автомобили, работающие на дизельном топливе.
    - *Hybrid*: автомобили, использующие как топливо, так и электричество.
    - *Electric*: полностью электрические автомобили.

- *Transmission*: тип трансмиссии:
    - *Manual*: механическая коробка передач.
    - *Automatic*: автоматическая коробка передач.
    - *Semi-Automatic*: полуавтоматическая коробка передач.

- *Mileage*: общее расстояние (в км), которое проехал автомобиль.

- *Doors*: количество дверей в автомобиле.

- *Owner_Count*: количество предыдущих владельцев автомобиля.

- *Price*: цена автомобиля (таргет).

In [71]:
import optuna as opt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing  import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

sns.set_theme()

## Знакомство с данными

In [72]:
data = pd.read_csv('../data/car_price_dataset.csv')
data

,Brand,Model,Year,Engine_Size,Fuel_Type,Transmission,Mileage,Doors,Owner_Count,Price
0,Kia,Rio,2020,4.2,Diesel,Manual,289944,3,5,8501
1,Chevrolet,Malibu,2012,2.0,Hybrid,Automatic,5356,2,3,12092
2,Mercedes,GLA,2020,4.2,Diesel,Automatic,231440,4,2,11171
3,Audi,Q5,2023,2.0,Electric,Manual,160971,2,1,11780
4,Volkswagen,Golf,2003,2.6,Hybrid,Semi-Automatic,286618,3,3,2867
...,...,...,...,...,...,...,...,...,...,...
9995,Kia,Optima,2004,3.7,Diesel,Semi-Automatic,5794,2,4,8884
9996,Chevrolet,Impala,2002,1.4,Electric,Automatic,168000,2,1,6240
9997,BMW,3 Series,2010,3.0,Petrol,Automatic,86664,5,1,9866
9998,Ford,Explorer,2002,1.4,Hybrid,Automatic,225772,4,1,4084


In [73]:
# проверим данные на наличие пропусков
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Brand         10000 non-null  object 
 1   Model         10000 non-null  object 
 2   Year          10000 non-null  int64  
 3   Engine_Size   10000 non-null  float64
 4   Fuel_Type     10000 non-null  object 
 5   Transmission  10000 non-null  object 
 6   Mileage       10000 non-null  int64  
 7   Doors         10000 non-null  int64  
 8   Owner_Count   10000 non-null  int64  
 9   Price         10000 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 781.4+ KB


In [74]:
# проверим на наличие выбросов числовые признаки
data.describe()

,Year,Engine_Size,Mileage,Doors,Owner_Count,Price
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000
mean,2011.543700,3.000560,149239.111800,3.497100,2.991100,8852.96440
std,6.897699,1.149324,86322.348957,1.110097,1.422682,3112.59681
min,2000.000000,1.000000,25.000000,2.000000,1.000000,2000.00000
25%,2006.000000,2.000000,74649.250000,3.000000,2.000000,6646.00000
50%,2012.000000,3.000000,149587.000000,3.000000,3.000000,8858.50000
75%,2017.000000,4.000000,223577.500000,4.000000,4.000000,11086.50000
max,2023.000000,5.000000,299947.000000,5.000000,5.000000,18301.00000
